In [ ]:
# https://www.linkedin.com/pulse/trends-recsys-two-towers-llms-vijay-raghavan-ph-d-m-b-a-/




import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class UserTowerRecommender(nn.Module):
    def __init__(self, user_feature_dim, hidden_dims, num_categories):
        super(UserTowerRecommender, self).__init__()
        
        # Define the user tower as a series of dense layers
        layers = []
        input_dim = user_feature_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.5))  # Add dropout for regularization
            input_dim = hidden_dim
        self.user_tower = nn.Sequential(*layers)
        
        # Final layer to produce predictions for each category
        self.output_layer = nn.Linear(input_dim, num_categories)

    def forward(self, user_features):
        user_out = self.user_tower(user_features)
        return self.output_layer(user_out)

# Example usage
user_feature_dim = 10  # Suppose each user is represented by 10 features
hidden_dims = [128, 64]  # Two hidden layers
num_categories = 5  # Predict interest in 5 categories

model = UserTowerRecommender(user_feature_dim, hidden_dims, num_categories)
user_features = torch.rand((1, user_feature_dim))
category_scores = model(user_features)

print(category_scores)





import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ItemTowerRecommender(nn.Module):
    def __init__(self, item_feature_dim, hidden_dims):
        super(ItemTowerRecommender, self).__init__()
        
        # Define the item tower as a series of dense layers
        layers = []
        input_dim = item_feature_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.5))  # Add dropout for regularization
            input_dim = hidden_dim
        self.item_tower = nn.Sequential(*layers)
        
        # Final layer to produce a score for the item's predicted popularity
        self.output_layer = nn.Linear(input_dim, 1)

    def forward(self, item_features):
        item_out = self.item_tower(item_features)
        return self.output_layer(item_out)

# Example usage
item_feature_dim = 8  # Suppose each item is represented by 8 features
hidden_dims = [64, 32]  # Two hidden layers

model = ItemTowerRecommender(item_feature_dim, hidden_dims)
item_features = torch.rand((1, item_feature_dim))
popularity_score = model(item_features)

print(popularity_score)


import numpy as np

# Sample user-item interaction matrix (ratings)
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

# Perform SVD
U, sigma, Vt = np.linalg.svd(R, full_matrices=False)

print("User embeddings (dense vectors):")
print(U)

print("\nItem embeddings (dense vectors):")
print(Vt.T)






import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

# Define the Neural Network
class ComplexDenseRepresentationNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, dense_dim):
        super(ComplexDenseRepresentationNetwork, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, dense_dim)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Hyperparameters
input_dim = 10
hidden_dim = 20
dense_dim = 5
batch_size = 5
learning_rate = 0.01
epochs = 2000

# Sample data: 10 one-hot encoded vectors representing 10 items
data = torch.eye(10)
targets = torch.rand(10, 5)

# Split data into training and validation sets (80% train, 20% validation)
dataset = CustomDataset(data, targets)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize the network
model = ComplexDenseRepresentationNetwork(input_dim, hidden_dim, dense_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop with Validation
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for batch_data, batch_targets in train_loader:
        outputs = model(batch_data)
        loss = criterion(outputs, batch_targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_data, batch_targets in val_loader:
            outputs = model(batch_data)
            loss = criterion(outputs, batch_targets)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    # Print losses every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Extract and print dense vectors
with torch.no_grad():
    dense_vectors = model(data)
    print(dense_vectors)





 import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ComplexContextualRecommender(nn.Module):
    def __init__(self, num_users, num_items, num_contexts, user_embed_dim, item_embed_dim, context_embed_dim):
        super(ComplexContextualRecommender, self).__init__()

        # Embedding layers with different dimensions
        self.user_embedding = nn.Embedding(num_users, user_embed_dim)
        self.item_embedding = nn.Embedding(num_items, item_embed_dim)
        self.context_embedding = nn.Embedding(num_contexts, context_embed_dim)

        # Calculate total embeddings dimension
        total_embed_dim = user_embed_dim + item_embed_dim + context_embed_dim
        
        # Dense layers with InstanceNorm and Dropout
        self.fc1 = nn.Linear(total_embed_dim, 256)
        self.in1 = nn.InstanceNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.in2 = nn.InstanceNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.in3 = nn.InstanceNorm1d(64)
        self.fc_out = nn.Linear(64, 1)  # Output a score
        self.dropout = nn.Dropout(0.5)

    def forward(self, user, item, context):
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)
        context_embed = self.context_embedding(context)

        # Concatenate the embeddings
        x = torch.cat([user_embed, item_embed, context_embed], 1)

        # Pass through dense layers with activations, instance normalization, and dropout
        x = self.dropout(F.relu(self.in1(self.fc1(x).unsqueeze(1)).squeeze(1)))
        x = self.dropout(F.relu(self.in2(self.fc2(x).unsqueeze(1)).squeeze(1)))
        x = self.dropout(F.relu(self.in3(self.fc3(x).unsqueeze(1)).squeeze(1)))
        x = self.fc_out(x)
        return x

# Hyperparameters and Data
num_users = 1000
num_items = 5000
num_contexts = 3
user_embed_dim = 50
item_embed_dim = 100
context_embed_dim = 10
batch_size = 32

# Initialize the complex model
model = ComplexContextualRecommender(num_users, num_items, num_contexts, user_embed_dim, item_embed_dim, context_embed_dim)
criterion = nn.MSELoss() # Assuming a regression problem
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate random sample data
users = torch.randint(0, num_users, (1000,))
items = torch.randint(0, num_items, (1000,))
contexts = torch.randint(0, num_contexts, (1000,))
ratings = torch.rand((1000, 1)) # Random ratings

# Sample Training loop for 100 epochs
for epoch in range(100):
    model.train()
    total_loss = 0.0

    # Create mini-batches
    for i in range(0, len(users), batch_size):
        batch_users = users[i:i+batch_size]
        batch_items = items[i:i+batch_size]
        batch_contexts = contexts[i:i+batch_size]
        batch_ratings = ratings[i:i+batch_size]

        # Skip the batch if size is 1
        if len(batch_users) == 1:
            continue

        optimizer.zero_grad()
        outputs = model(batch_users, batch_items, batch_contexts)
        loss = criterion(outputs, batch_ratings)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch+1) % 10 == 0:
        avg_loss = total_loss / (len(users) // batch_size)
        print(f'Epoch [{epoch+1}/100], Loss: {avg_loss:.4f}')


# Example prediction
user = torch.tensor([5])
item = torch.tensor([100])
context = torch.tensor([2])
score = model(user, item, context)
print(f"Predicted Score: {score.item():.4f}")

In [1]:
import random
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

# VARIABLES
MY_SEED = 42
TEST_SIZE = 0.20
DIR_DATA_LIMPIA = '../datos/data-limpia'

# SEMILLA
random.seed(MY_SEED)
np.random.seed(MY_SEED)

# IMPORTAR DATA
catalogo = pd.read_csv(f"{DIR_DATA_LIMPIA}/catalogo.csv", sep=",", encoding="latin1")
mf_dataset = pd.read_csv(f"{DIR_DATA_LIMPIA}/mf_dataset.csv", sep=",", encoding="latin1")
catalogo.head()

2024-10-19 05:56:27.909894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 05:56:27.934456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 05:56:27.944786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 05:56:27.970059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-19 05:56:29.633416: W tensorflow/compiler/tf2

,oid,nombre,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,hito,skill,knowledge,complexity_8bits,complexity_12bits
0,0973dae0e1b74ab8baa8d94339ee3ae6,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,1,0,1,1,1,1,1,7,23,279
1,16f619db31204ded9418136c4587ddd8,Calculadora GeomÃ©trica,0,0,1,0,0,0,0,1,0,0,0,1,2,1,1,17,529
2,17022c9ceac94ec5b2e7bc934c7b2d6f,Subsecuencias de ADN,0,0,1,0,0,0,1,1,0,1,1,1,2,3,7,55,567
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257


# DATOS

In [2]:
ratio_de_ejercicios = mf_dataset.sum().values.tolist()[1:]
resultados = [ n / len(mf_dataset) for n in ratio_de_ejercicios ]
max_valor = max(resultados)
indice_valor = resultados.index(max_valor)

print(f'Cantidad de registros de alumnos: {len(mf_dataset)}')
print(f'Cantidad de veces hechos (Ejercicios): {ratio_de_ejercicios}')
print(f'Porcentaje de realizacion del ejercicio: {resultados}')
print()
print(f'Indice del ejercicio: {indice_valor}\nCantidad del Ratio: {ratio_de_ejercicios[indice_valor]}\nRatio Max %: {max_valor}\n')

catalogo['ratio'] = resultados
catalogo.head()

Cantidad de registros de alumnos: 766
Cantidad de veces hechos (Ejercicios): [603, 275, 17, 542, 733, 0, 277, 195, 21, 42, 599, 38, 69, 233, 0, 0, 0, 737, 389, 0, 0, 37, 650, 590, 0, 730, 678, 0, 168, 653, 0, 4, 395, 11, 0, 347, 0, 184, 87, 0, 0, 0, 472, 0, 314, 0, 0, 0, 4, 0, 0, 0, 0]
Porcentaje de realizacion del ejercicio: [0.7872062663185379, 0.35900783289817234, 0.022193211488250653, 0.7075718015665796, 0.9569190600522193, 0.0, 0.36161879895561355, 0.2545691906005222, 0.02741514360313316, 0.05483028720626632, 0.7819843342036553, 0.04960835509138381, 0.09007832898172324, 0.304177545691906, 0.0, 0.0, 0.0, 0.9621409921671018, 0.5078328981723238, 0.0, 0.0, 0.048302872062663184, 0.8485639686684073, 0.7702349869451697, 0.0, 0.9530026109660574, 0.8851174934725848, 0.0, 0.2193211488250653, 0.8524804177545692, 0.0, 0.005221932114882507, 0.5156657963446475, 0.014360313315926894, 0.0, 0.45300261096605743, 0.0, 0.2402088772845953, 0.11357702349869452, 0.0, 0.0, 0.0, 0.6161879895561357, 0.0, 0

,oid,nombre,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,hito,skill,knowledge,complexity_8bits,complexity_12bits,ratio
0,0973dae0e1b74ab8baa8d94339ee3ae6,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,1,0,1,1,1,1,1,7,23,279,0.787206
1,16f619db31204ded9418136c4587ddd8,Calculadora GeomÃ©trica,0,0,1,0,0,0,0,1,0,0,0,1,2,1,1,17,529,0.359008
2,17022c9ceac94ec5b2e7bc934c7b2d6f,Subsecuencias de ADN,0,0,1,0,0,0,1,1,0,1,1,1,2,3,7,55,567,0.022193
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259,0.707572
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257,0.956919


In [3]:
# Asignar pesos a cada componente
w_hito = 0.4
w_skill = 0.25
w_knowledge = 0.25
w_popularidad = 0.10

catalogo['combined_score_1'] = (
    w_hito * (catalogo['hito'] / 4) +
    (w_skill+w_knowledge) * (catalogo['complexity_8bits'] / 255) + 
    w_popularidad * catalogo['ratio']
)

catalogo['combined_score_2'] = (
    (w_hito+w_skill+w_knowledge) * (catalogo['complexity_12bits'] / 2303) + 
    w_popularidad * catalogo['ratio']
)

catalogo['combined_score_3'] = (
    w_hito * (catalogo['hito'] / 4) +
    w_skill * (catalogo['skill'] / 15) +
    w_knowledge * (catalogo['knowledge'] / 15) +
    w_popularidad * catalogo['ratio']
)

catalogo.head()

,oid,nombre,h4,h3,h2,h1,s4,s3,s2,s1,...,k1,hito,skill,knowledge,complexity_8bits,complexity_12bits,ratio,combined_score_1,combined_score_2,combined_score_3
0,0973dae0e1b74ab8baa8d94339ee3ae6,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,1,...,1,1,1,7,23,279,0.787206,0.223819,0.187752,0.312054
1,16f619db31204ded9418136c4587ddd8,Calculadora GeomÃ©trica,0,0,1,0,0,0,0,1,...,1,2,1,1,17,529,0.359008,0.269234,0.242631,0.269234
2,17022c9ceac94ec5b2e7bc934c7b2d6f,Subsecuencias de ADN,0,0,1,0,0,0,1,1,...,1,2,3,7,55,567,0.022193,0.310062,0.223800,0.368886
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,0,0,0,1,0,0,0,0,...,1,1,0,3,3,259,0.707572,0.176640,0.171973,0.220757
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,...,1,1,0,1,1,257,0.956919,0.197653,0.196126,0.212359


In [4]:
n_comp_8bits = MinMaxScaler()
catalogo['comp_8bits_norm'] = n_comp_8bits.fit_transform(catalogo[['complexity_8bits']])

n_comp_12bits = MinMaxScaler()
catalogo['comp_12bits_norm'] = n_comp_12bits.fit_transform(catalogo[['complexity_12bits']])

catalogo.head()

,oid,nombre,h4,h3,h2,h1,s4,s3,s2,s1,...,skill,knowledge,complexity_8bits,complexity_12bits,ratio,combined_score_1,combined_score_2,combined_score_3,comp_8bits_norm,comp_12bits_norm
0,0973dae0e1b74ab8baa8d94339ee3ae6,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,1,...,1,7,23,279,0.787206,0.223819,0.187752,0.312054,0.111111,0.011055
1,16f619db31204ded9418136c4587ddd8,Calculadora GeomÃ©trica,0,0,1,0,0,0,0,1,...,1,1,17,529,0.359008,0.269234,0.242631,0.269234,0.080808,0.136683
2,17022c9ceac94ec5b2e7bc934c7b2d6f,Subsecuencias de ADN,0,0,1,0,0,0,1,1,...,3,7,55,567,0.022193,0.310062,0.223800,0.368886,0.272727,0.155779
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,0,0,0,1,0,0,0,0,...,0,3,3,259,0.707572,0.176640,0.171973,0.220757,0.010101,0.001005
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,...,0,1,1,257,0.956919,0.197653,0.196126,0.212359,0.000000,0.000000


In [5]:
df_catalogo = catalogo[['oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity_8bits', 'complexity_12bits', 'combined_score_1', 'combined_score_2', 'combined_score_3']]
df_catalogo.head()

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
0,0973dae0e1b74ab8baa8d94339ee3ae6,0,0,0,1,0,0,0,1,0,1,1,1,23,279,0.223819,0.187752,0.312054
1,16f619db31204ded9418136c4587ddd8,0,0,1,0,0,0,0,1,0,0,0,1,17,529,0.269234,0.242631,0.269234
2,17022c9ceac94ec5b2e7bc934c7b2d6f,0,0,1,0,0,0,1,1,0,1,1,1,55,567,0.310062,0.223800,0.368886
3,171b5e86d4fb47268f2692587fbec073,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.176640,0.171973,0.220757
4,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.197653,0.196126,0.212359


In [6]:
def factorization_to_ratings(df_exercises: pd.DataFrame, df_matrix: pd.DataFrame)-> pd.DataFrame:
    rows = []
    for _, row_matrix in df_matrix.iterrows():
        user_id = row_matrix['rut']
        for exercise in df_matrix.columns[1:]:
            if row_matrix[exercise] == 1:
                row_exercise = df_exercises.iloc[int(exercise.lstrip('e'))]
                new_row = {'rut': user_id}
                new_row.update(row_exercise.to_dict())
                rows.append(new_row)
    df = pd.DataFrame(rows)      
    return df

In [7]:
exercises_users = factorization_to_ratings(df_catalogo, mf_dataset)
exercises_users.head(10)

,rut,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
0,0,0973dae0e1b74ab8baa8d94339ee3ae6,0,0,0,1,0,0,0,1,0,1,1,1,23,279,0.223819,0.187752,0.312054
1,0,16f619db31204ded9418136c4587ddd8,0,0,1,0,0,0,0,1,0,0,0,1,17,529,0.269234,0.242631,0.269234
2,0,171b5e86d4fb47268f2692587fbec073,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.176640,0.171973,0.220757
3,0,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.197653,0.196126,0.212359
4,0,29f15ef8dc32426f945f64e28c910a57,0,0,1,0,0,0,0,1,0,0,1,1,19,531,0.273417,0.243674,0.302829
5,0,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.184081,0.179414,0.228198
6,0,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,2,258,0.200136,0.197039,0.229547
7,0,729d37da8f2d46f3af2d891df04949ef,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.156666,0.151999,0.200783
8,0,7f60644b0a1b484681ae5c8e36166c58,0,0,0,1,0,0,0,1,0,0,0,1,17,273,0.218190,0.191543,0.218190
9,0,80c61dae74fa4915bf272ab17dfa62ff,0,0,0,1,0,0,0,0,0,0,1,0,2,258,0.180945,0.177849,0.210357


In [8]:
# Dividir en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(exercises_users, test_size=TEST_SIZE, random_state=MY_SEED)
train_data.head(10)

,rut,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
1197,89,52620b0c858a4c59bc324b65278d28bd,0,0,0,1,0,0,0,1,0,0,1,1,19,275,0.167673,0.137886,0.197084
8052,1099,7f60644b0a1b484681ae5c8e36166c58,0,0,0,1,0,0,0,1,0,0,0,1,17,273,0.218190,0.191543,0.218190
5461,403,0973dae0e1b74ab8baa8d94339ee3ae6,0,0,0,1,0,0,0,1,0,1,1,1,23,279,0.223819,0.187752,0.312054
7389,1026,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.127418,0.125891,0.142124
6478,923,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.184081,0.179414,0.228198
1429,104,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,2,258,0.200136,0.197039,0.229547
4464,336,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.127418,0.125891,0.142124
4092,307,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.197653,0.196126,0.212359
2072,153,ac7382763e484d37908da54c076f7577,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.157449,0.152782,0.201567
8660,1159,729d37da8f2d46f3af2d891df04949ef,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.156666,0.151999,0.200783


# DIVISION DE DATOS

In [9]:
# COMLEXITY 8 BITS
drop_columns = ['rut', 'oid', 'complexity_12bits', 'combined_score_1', 'combined_score_2', 'combined_score_3']

x_train_complexity_8bits = train_data.drop(drop_columns, axis=1).values
y_train_complexity_8bits = train_data['rut'].values

x_test_complexity_8bits = test_data.drop(drop_columns, axis=1).values
y_test_complexity_8bits = test_data['rut'].values

train_data.drop(drop_columns, axis=1).head()

,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits
1197,0,0,0,1,0,0,0,1,0,0,1,1,19
8052,0,0,0,1,0,0,0,1,0,0,0,1,17
5461,0,0,0,1,0,0,0,1,0,1,1,1,23
7389,0,0,0,1,0,0,0,0,0,0,0,1,1
6478,0,0,0,1,0,0,0,0,0,0,1,1,3


In [10]:
# COMLEXITY 12 BITS
drop_columns = ['rut', 'oid', 'complexity_8bits', 'combined_score_1', 'combined_score_2', 'combined_score_3']

x_train_complexity_12bits = train_data.drop(drop_columns, axis=1).values
y_train_complexity_12bits = train_data['rut'].values

x_test_complexity_12bits = test_data.drop(drop_columns, axis=1).values
y_test_complexity_12bits = test_data['rut'].values

train_data.drop(drop_columns, axis=1).head()

,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_12bits
1197,0,0,0,1,0,0,0,1,0,0,1,1,275
8052,0,0,0,1,0,0,0,1,0,0,0,1,273
5461,0,0,0,1,0,0,0,1,0,1,1,1,279
7389,0,0,0,1,0,0,0,0,0,0,0,1,257
6478,0,0,0,1,0,0,0,0,0,0,1,1,259


In [11]:
# COMBINED SCORE 1
drop_columns = ['rut', 'oid', 'complexity_8bits', 'complexity_12bits', 'combined_score_2', 'combined_score_3']

x_train_combined_score_1 = train_data.drop(drop_columns, axis=1).values
y_train_combined_score_1 = train_data['rut'].values

x_test_combined_score_1 = test_data.drop(drop_columns, axis=1).values
y_test_combined_score_1 = test_data['rut'].values

train_data.drop(drop_columns, axis=1).head()

,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,combined_score_1
1197,0,0,0,1,0,0,0,1,0,0,1,1,0.167673
8052,0,0,0,1,0,0,0,1,0,0,0,1,0.218190
5461,0,0,0,1,0,0,0,1,0,1,1,1,0.223819
7389,0,0,0,1,0,0,0,0,0,0,0,1,0.127418
6478,0,0,0,1,0,0,0,0,0,0,1,1,0.184081


In [12]:
# COMBINED SCORE 2
drop_columns = ['rut', 'oid', 'complexity_8bits', 'complexity_12bits', 'combined_score_1', 'combined_score_3']

x_train_combined_score_2 = train_data.drop(drop_columns, axis=1).values
y_train_combined_score_2 = train_data['rut'].values

x_test_combined_score_2 = test_data.drop(drop_columns, axis=1).values
y_test_combined_score_2 = test_data['rut'].values

train_data.drop(drop_columns, axis=1).head()

,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,combined_score_2
1197,0,0,0,1,0,0,0,1,0,0,1,1,0.137886
8052,0,0,0,1,0,0,0,1,0,0,0,1,0.191543
5461,0,0,0,1,0,0,0,1,0,1,1,1,0.187752
7389,0,0,0,1,0,0,0,0,0,0,0,1,0.125891
6478,0,0,0,1,0,0,0,0,0,0,1,1,0.179414


In [13]:
# COMBINED SCORE 2
drop_columns = ['rut', 'oid', 'complexity_8bits', 'complexity_12bits', 'combined_score_1', 'combined_score_2']

x_train_combined_score_3 = train_data.drop(drop_columns, axis=1).values
y_train_combined_score_3 = train_data['rut'].values

x_test_combined_score_3 = test_data.drop(drop_columns, axis=1).values
y_test_combined_score_3 = test_data['rut'].values

train_data.drop(drop_columns, axis=1).head()

,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,combined_score_3
1197,0,0,0,1,0,0,0,1,0,0,1,1,0.197084
8052,0,0,0,1,0,0,0,1,0,0,0,1,0.218190
5461,0,0,0,1,0,0,0,1,0,1,1,1,0.312054
7389,0,0,0,1,0,0,0,0,0,0,0,1,0.142124
6478,0,0,0,1,0,0,0,0,0,0,1,1,0.228198


# MODELOS

In [14]:
model_complexity_8bits = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_complexity_8bits.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear')
])

model_complexity_8bits.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
model_complexity_8bits.summary()

history_complexity_8bits = model_complexity_8bits.fit(x_train_complexity_8bits, y_train_complexity_8bits, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

print()
loss_complexity_8bits = model_complexity_8bits.evaluate(x_test_complexity_8bits, y_test_complexity_8bits)
print(f'Loss en el conjunto de prueba: {loss_complexity_8bits}')
predictions_complexity_8bits = model_complexity_8bits.predict(x_test_complexity_8bits)

2024-10-19 05:56:33.435762: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 493217.7188 - val_loss: 364034.7188
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 370687.4688 - val_loss: 276371.0938
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 272096.7500 - val_loss: 192501.9219
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 205748.4062 - val_loss: 181839.8125
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 196169.0156 - val_loss: 181363.0625
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 192052.1719 - val_loss: 181168.6094
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 192814.9219 - val_loss: 180131.4062
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 192505.3906 - val_loss: 179815.3750
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 190742.9062 - val_loss: 179304.5781
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 190967.8906 - val_loss: 179231.5469
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

In [15]:
model_complexity_12bits = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_complexity_12bits.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear')
])

model_complexity_12bits.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
model_complexity_12bits.summary()

history_complexity_12bits = model_complexity_12bits.fit(x_train_complexity_12bits, y_train_complexity_12bits, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

print()
loss_complexity_12bits = model_complexity_12bits.evaluate(x_test_complexity_12bits, y_test_complexity_12bits)
print(f'Loss en el conjunto de prueba: {loss_complexity_12bits}')
predictions_complexity_12bits = model_complexity_12bits.predict(x_test_complexity_12bits)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 342238.5312 - val_loss: 256869.3750
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 270757.0312 - val_loss: 255871.2344
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 273316.8125 - val_loss: 254706.9844
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 268932.3750 - val_loss: 252607.8750
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 267645.4062 - val_loss: 250119.0312
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 263612.4062 - val_loss: 247245.8906
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 252638.0781 - val_loss: 245700.0000
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 255705.0781 - val_loss: 236075.6094
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 245879.6562 - val_loss: 223141.8281
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 234255.6562 - val_loss: 208805.9844
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

In [16]:
model_combined_score_1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_combined_score_1.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear')
])

model_combined_score_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
model_combined_score_1.summary()

history_combined_score_1 = model_combined_score_1.fit(x_train_combined_score_1, y_train_combined_score_1, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

print()
loss_combined_score_1 = model_combined_score_1.evaluate(x_test_combined_score_1, y_test_combined_score_1)
print(f'Loss en el conjunto de prueba: {loss_combined_score_1}')
predictions_combined_score_1 = model_combined_score_1.predict(x_test_combined_score_1)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 486665.2812 - val_loss: 333782.7188
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 286046.8438 - val_loss: 189486.2812
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 198016.1875 - val_loss: 181827.3750
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 191776.4062 - val_loss: 179526.8281
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 186245.0938 - val_loss: 178757.2188
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 190552.7969 - val_loss: 178340.0000
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 190397.7344 - val_loss: 177786.9062
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 189012.0156 - val_loss: 177365.2031
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 182181.3594 - val_loss: 177116.4375
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 186371.8281 - val_loss: 177040.8438
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

In [17]:
model_combined_score_2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_combined_score_2.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear')
])

model_combined_score_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
model_combined_score_2.summary()

history_combined_score_2 = model_combined_score_2.fit(x_train_combined_score_2, y_train_combined_score_2, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

print()
loss_combined_score_2 = model_combined_score_2.evaluate(x_test_combined_score_2, y_test_combined_score_2)
print(f'Loss en el conjunto de prueba: {loss_combined_score_2}')
predictions_combined_score_2 = model_combined_score_2.predict(x_test_combined_score_2)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 489583.0938 - val_loss: 361133.0938
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 304795.6875 - val_loss: 191633.0469
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 199902.2500 - val_loss: 183038.7344
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 197579.6562 - val_loss: 180243.3438
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 192256.3906 - val_loss: 179490.0781
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 191676.4375 - val_loss: 178687.3281
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 191220.0625 - val_loss: 178370.9844
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 189125.4531 - val_loss: 177832.8438
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 189673.3438 - val_loss: 177518.1406
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 187130.7969 - val_loss: 177315.1875
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

In [18]:
model_combined_score_3 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_combined_score_3.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear')
])

model_combined_score_3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
model_combined_score_3.summary()

history_combined_score_3 = model_combined_score_3.fit(x_train_combined_score_3, y_train_combined_score_3, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

print()
loss_combined_score_3 = model_combined_score_3.evaluate(x_test_combined_score_3, y_test_combined_score_3)
print(f'Loss en el conjunto de prueba: {loss_combined_score_3}')
predictions_combined_score_3 = model_combined_score_3.predict(x_test_combined_score_3)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 494938.7812 - val_loss: 329539.9062
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 277484.0625 - val_loss: 190272.6719
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 201134.3125 - val_loss: 182483.0781
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 193518.4531 - val_loss: 180142.8281
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 192865.4219 - val_loss: 179132.6562
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 192801.4375 - val_loss: 178536.6562
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 185942.2656 - val_loss: 178293.4844
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 189488.2500 - val_loss: 177756.3125
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 186114.8906 - val_loss: 177424.0156
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 185622.1094 - val_loss: 177142.2969
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━

# PREDICCIONES

In [19]:
# random_user_id = mf_dataset.sample(1)['rut'].values[0]
# items_interacted = exercises_users[exercises_users['rut'] == random_user_id]['oid'].values

# print(f'Rut Estudiante: {random_user_id}')
# print(f'Ejercicios realizados por el usuario [ {random_user_id} ]')

# catalogo[catalogo['oid'].isin(items_interacted)].sort_values(by='complexity_12bits', ascending=True)

#### ESTUDIANTES

In [20]:
estudiante_1 = 1305
estudiante_2 = 985

In [21]:
ejercicios_realizados_est_1 = exercises_users[exercises_users['rut'] == estudiante_1]['oid'].values
ejercicios_realizados_est_2 = exercises_users[exercises_users['rut'] == estudiante_2]['oid'].values

In [37]:
catalogo[catalogo['oid'].isin(ejercicios_realizados_est_1)].sort_values(by='complexity_12bits', ascending=True)[['oid', 'nombre', 'skill', 'knowledge']]

,oid,nombre,skill,knowledge
25,89f44e7f5842479fb283e43c52ce067b,Ordenar tres nÃºmeros,0,1
23,80c61dae74fa4915bf272ab17dfa62ff,Conversor de Decimal a Binario,0,2


In [38]:
catalogo[catalogo['oid'].isin(ejercicios_realizados_est_2)].sort_values(by='complexity_12bits', ascending=True)[['oid', 'nombre', 'skill', 'knowledge']]

,oid,nombre,skill,knowledge
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,1
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,0,1
25,89f44e7f5842479fb283e43c52ce067b,Ordenar tres nÃºmeros,0,1
17,718578451f3f4eca87437cadfe98d688,Suma de los N primeros nÃºmeros naturales,0,2
23,80c61dae74fa4915bf272ab17dfa62ff,Conversor de Decimal a Binario,0,2
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,0,3
10,46850a246d48484b8f104f8aab5679b6,Descomponer un nÃºmero,0,3
18,729d37da8f2d46f3af2d891df04949ef,Juego Adivina mi nÃºmero,0,3
26,8f24397e36034cccb71e9d578975c33d,Contestador AutomÃ¡tico,0,3
29,a3963220090f4e50a266ce53d33b9841,AprobaciÃ³n de CrÃ©ditos,0,3


In [22]:
ejercicios_no_realizados_1 = catalogo[~catalogo['oid'].isin(ejercicios_realizados_est_1)]
ejercicios_no_realizados_2 = catalogo[~catalogo['oid'].isin(ejercicios_realizados_est_2)]

In [23]:
# ESTUDIANTE 1
predicciones_estudiante_1_complexity_8bits = model_complexity_8bits.predict(ejercicios_no_realizados_1.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_12bits', 'combined_score_1', 'combined_score_2', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_1_complexity_12bits = model_complexity_12bits.predict(ejercicios_no_realizados_1.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'combined_score_1', 'combined_score_2', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_1_combined_score_1 = model_combined_score_1.predict(ejercicios_no_realizados_1.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'complexity_12bits', 'combined_score_2', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_1_combined_score_2 = model_combined_score_2.predict(ejercicios_no_realizados_1.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'complexity_12bits', 'combined_score_1', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_1_combined_score_3 = model_combined_score_3.predict(ejercicios_no_realizados_1.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'complexity_12bits', 'combined_score_1', 'combined_score_2', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [24]:
# ESTUDIANTE 2
predicciones_estudiante_2_complexity_8bits = model_complexity_8bits.predict(ejercicios_no_realizados_2.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_12bits', 'combined_score_1', 'combined_score_2', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_2_complexity_12bits = model_complexity_12bits.predict(ejercicios_no_realizados_2.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'combined_score_1', 'combined_score_2', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_2_combined_score_1 = model_combined_score_1.predict(ejercicios_no_realizados_2.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'complexity_12bits', 'combined_score_2', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_2_combined_score_2 = model_combined_score_2.predict(ejercicios_no_realizados_2.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'complexity_12bits', 'combined_score_1', 'combined_score_3', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()
predicciones_estudiante_2_combined_score_3 = model_combined_score_3.predict(ejercicios_no_realizados_2.drop(labels=['oid', 'nombre', 'hito', 'skill', 'knowledge', 'ratio', 'complexity_8bits', 'complexity_12bits', 'combined_score_1', 'combined_score_2', 'comp_8bits_norm', 'comp_12bits_norm'], axis=1).values).flatten()



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


### ESTUDIANTE 1

In [25]:
rec_complexity_8bits = ejercicios_no_realizados_1.copy()
rec_complexity_8bits['predicted_preference'] = predicciones_estudiante_1_complexity_8bits
rec_complexity_8bits = rec_complexity_8bits.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_complexity_8bits['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
0,0973dae0e1b74ab8baa8d94339ee3ae6,0,0,0,1,0,0,0,1,0,1,1,1,23,279,0.223819,0.187752,0.312054
4,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.197653,0.196126,0.212359
5,24b87304a1eb4e95856da7f574a66fe8,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
7,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.127418,0.125891,0.142124
17,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,2,258,0.200136,0.197039,0.229547
22,7f60644b0a1b484681ae5c8e36166c58,0,0,0,1,0,0,0,1,0,0,0,1,17,273,0.218190,0.191543,0.218190
30,abbcd12771fd45ca823f6222f72b48c8,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
36,be17b556785c428e8dabbef0c9d11f6f,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
47,e6093c934b8c40b4bee9969b32a30bed,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
51,eb91ed4e997b4ca98f3c86e3e86a3755,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667


In [26]:
rec_complexity_12bits = ejercicios_no_realizados_1.copy()
rec_complexity_12bits['predicted_preference'] = predicciones_estudiante_1_complexity_12bits
rec_complexity_12bits = rec_complexity_12bits.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_complexity_12bits['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
5,24b87304a1eb4e95856da7f574a66fe8,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
16,677fbf669a0d496c843190c116b4f173,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333
24,82e092a2744544d1b6cedde23d4596a5,0,1,0,0,0,0,0,1,1,1,1,1,31,1055,0.360784,0.412288,0.566667
30,abbcd12771fd45ca823f6222f72b48c8,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
36,be17b556785c428e8dabbef0c9d11f6f,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
39,cb796b16ddf1435da29b223fc084c847,0,1,0,0,0,0,0,1,1,1,1,1,31,1055,0.360784,0.412288,0.566667
47,e6093c934b8c40b4bee9969b32a30bed,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
50,e9df502fccd9455a8a7609d542a857c8,1,0,0,0,0,0,0,1,0,0,1,1,19,2067,0.437255,0.807772,0.466667
51,eb91ed4e997b4ca98f3c86e3e86a3755,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
52,f7f404137f974160bcbfa423f5d35eda,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333


In [27]:
rec_combined_score_1 = ejercicios_no_realizados_1.copy()
rec_combined_score_1['predicted_preference'] = predicciones_estudiante_1_combined_score_1
rec_combined_score_1 = rec_combined_score_1.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_combined_score_1['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
0,0973dae0e1b74ab8baa8d94339ee3ae6,0,0,0,1,0,0,0,1,0,1,1,1,23,279,0.223819,0.187752,0.312054
3,171b5e86d4fb47268f2692587fbec073,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.176640,0.171973,0.220757
4,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.197653,0.196126,0.212359
7,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.127418,0.125891,0.142124
10,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.184081,0.179414,0.228198
17,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,2,258,0.200136,0.197039,0.229547
22,7f60644b0a1b484681ae5c8e36166c58,0,0,0,1,0,0,0,1,0,0,0,1,17,273,0.218190,0.191543,0.218190
26,8f24397e36034cccb71e9d578975c33d,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.194394,0.189728,0.238512
29,a3963220090f4e50a266ce53d33b9841,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.191130,0.186464,0.235248
42,d8395f43e4a1454d90346ac5a1ba561a,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.167501,0.162835,0.211619


In [28]:
rec_combined_score_2 = ejercicios_no_realizados_1.copy()
rec_combined_score_2['predicted_preference'] = predicciones_estudiante_1_combined_score_2
rec_combined_score_2 = rec_combined_score_2.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_combined_score_2['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
0,0973dae0e1b74ab8baa8d94339ee3ae6,0,0,0,1,0,0,0,1,0,1,1,1,23,279,0.223819,0.187752,0.312054
3,171b5e86d4fb47268f2692587fbec073,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.176640,0.171973,0.220757
4,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.197653,0.196126,0.212359
7,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.127418,0.125891,0.142124
10,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.184081,0.179414,0.228198
17,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,2,258,0.200136,0.197039,0.229547
22,7f60644b0a1b484681ae5c8e36166c58,0,0,0,1,0,0,0,1,0,0,0,1,17,273,0.218190,0.191543,0.218190
26,8f24397e36034cccb71e9d578975c33d,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.194394,0.189728,0.238512
29,a3963220090f4e50a266ce53d33b9841,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.191130,0.186464,0.235248
42,d8395f43e4a1454d90346ac5a1ba561a,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.167501,0.162835,0.211619


In [29]:
rec_combined_score_3 = ejercicios_no_realizados_1.copy()
rec_combined_score_3['predicted_preference'] = predicciones_estudiante_1_combined_score_3
rec_combined_score_3 = rec_combined_score_3.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_combined_score_3['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
0,0973dae0e1b74ab8baa8d94339ee3ae6,0,0,0,1,0,0,0,1,0,1,1,1,23,279,0.223819,0.187752,0.312054
3,171b5e86d4fb47268f2692587fbec073,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.176640,0.171973,0.220757
4,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.197653,0.196126,0.212359
7,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,1,257,0.127418,0.125891,0.142124
10,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.184081,0.179414,0.228198
17,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,2,258,0.200136,0.197039,0.229547
22,7f60644b0a1b484681ae5c8e36166c58,0,0,0,1,0,0,0,1,0,0,0,1,17,273,0.218190,0.191543,0.218190
26,8f24397e36034cccb71e9d578975c33d,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.194394,0.189728,0.238512
29,a3963220090f4e50a266ce53d33b9841,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.191130,0.186464,0.235248
42,d8395f43e4a1454d90346ac5a1ba561a,0,0,0,1,0,0,0,0,0,0,1,1,3,259,0.167501,0.162835,0.211619


### ESTUDIANTE 2

In [30]:
rec_complexity_8bits = ejercicios_no_realizados_2.copy()
rec_complexity_8bits['predicted_preference'] = predicciones_estudiante_2_complexity_8bits
rec_complexity_8bits = rec_complexity_8bits.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_complexity_8bits['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
2,17022c9ceac94ec5b2e7bc934c7b2d6f,0,0,1,0,0,0,1,1,0,1,1,1,55,567,0.310062,0.223800,0.368886
5,24b87304a1eb4e95856da7f574a66fe8,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
8,37e4f4a1e8174e9496d21b00d67fc8f1,0,0,1,0,0,0,1,1,0,1,1,1,55,567,0.310585,0.224322,0.369408
30,abbcd12771fd45ca823f6222f72b48c8,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
36,be17b556785c428e8dabbef0c9d11f6f,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
40,d28eb11da0a44881be979e551d122eb9,0,1,0,0,0,1,0,1,1,1,1,1,95,1119,0.486275,0.437299,0.633333
43,d849ec19c3834c2b9c9ff2ab34d2f3e3,0,1,0,0,0,0,1,1,1,1,1,1,63,1087,0.423529,0.424794,0.600000
46,dd1000315e5b43ac89ae1d416400d9d6,0,1,0,0,0,0,1,1,1,1,1,1,63,1087,0.423529,0.424794,0.600000
47,e6093c934b8c40b4bee9969b32a30bed,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
51,eb91ed4e997b4ca98f3c86e3e86a3755,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667


In [31]:
rec_complexity_12bits = ejercicios_no_realizados_2.copy()
rec_complexity_12bits['predicted_preference'] = predicciones_estudiante_2_complexity_12bits
rec_complexity_12bits = rec_complexity_12bits.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_complexity_12bits['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
5,24b87304a1eb4e95856da7f574a66fe8,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
16,677fbf669a0d496c843190c116b4f173,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333
24,82e092a2744544d1b6cedde23d4596a5,0,1,0,0,0,0,0,1,1,1,1,1,31,1055,0.360784,0.412288,0.566667
30,abbcd12771fd45ca823f6222f72b48c8,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
36,be17b556785c428e8dabbef0c9d11f6f,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
39,cb796b16ddf1435da29b223fc084c847,0,1,0,0,0,0,0,1,1,1,1,1,31,1055,0.360784,0.412288,0.566667
47,e6093c934b8c40b4bee9969b32a30bed,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
50,e9df502fccd9455a8a7609d542a857c8,1,0,0,0,0,0,0,1,0,0,1,1,19,2067,0.437255,0.807772,0.466667
51,eb91ed4e997b4ca98f3c86e3e86a3755,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
52,f7f404137f974160bcbfa423f5d35eda,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333


In [32]:
rec_combined_score_1 = ejercicios_no_realizados_2.copy()
rec_combined_score_1['predicted_preference'] = predicciones_estudiante_2_combined_score_1
rec_combined_score_1 = rec_combined_score_1.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_combined_score_1['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
5,24b87304a1eb4e95856da7f574a66fe8,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
16,677fbf669a0d496c843190c116b4f173,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333
27,995f2cf9681542a4a4738de074275f4a,0,1,0,0,0,0,1,0,0,1,1,1,39,1063,0.376471,0.415415,0.450000
34,b5e026a9cf014d71b110c58108a21752,0,1,0,0,0,0,0,0,1,1,0,0,12,1036,0.323529,0.404863,0.500000
36,be17b556785c428e8dabbef0c9d11f6f,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
41,d44d5fead16c47a986287ff2bc48d20f,0,1,0,0,0,0,1,0,1,1,1,1,47,1071,0.392157,0.418541,0.583333
43,d849ec19c3834c2b9c9ff2ab34d2f3e3,0,1,0,0,0,0,1,1,1,1,1,1,63,1087,0.423529,0.424794,0.600000
46,dd1000315e5b43ac89ae1d416400d9d6,0,1,0,0,0,0,1,1,1,1,1,1,63,1087,0.423529,0.424794,0.600000
49,e8a9dc7b6127482987d9b6dbca73ec5e,0,1,0,0,0,0,1,0,1,1,1,1,47,1071,0.392157,0.418541,0.583333
52,f7f404137f974160bcbfa423f5d35eda,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333


In [33]:
rec_combined_score_2 = ejercicios_no_realizados_2.copy()
rec_combined_score_2['predicted_preference'] = predicciones_estudiante_2_combined_score_2
rec_combined_score_2 = rec_combined_score_2.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_combined_score_2['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
5,24b87304a1eb4e95856da7f574a66fe8,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
16,677fbf669a0d496c843190c116b4f173,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333
30,abbcd12771fd45ca823f6222f72b48c8,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
34,b5e026a9cf014d71b110c58108a21752,0,1,0,0,0,0,0,0,1,1,0,0,12,1036,0.323529,0.404863,0.500000
36,be17b556785c428e8dabbef0c9d11f6f,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
47,e6093c934b8c40b4bee9969b32a30bed,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
49,e8a9dc7b6127482987d9b6dbca73ec5e,0,1,0,0,0,0,1,0,1,1,1,1,47,1071,0.392157,0.418541,0.583333
50,e9df502fccd9455a8a7609d542a857c8,1,0,0,0,0,0,0,1,0,0,1,1,19,2067,0.437255,0.807772,0.466667
51,eb91ed4e997b4ca98f3c86e3e86a3755,1,0,0,0,1,1,0,0,0,1,1,1,199,2247,0.790196,0.878116,0.716667
52,f7f404137f974160bcbfa423f5d35eda,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333


In [34]:
rec_combined_score_3 = ejercicios_no_realizados_2.copy()
rec_combined_score_3['predicted_preference'] = predicciones_estudiante_2_combined_score_3
rec_combined_score_3 = rec_combined_score_3.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['oid'].isin(rec_combined_score_3['oid'].values)] # .sort_values(by='complexity_12bits', ascending=True)

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,complexity_8bits,complexity_12bits,combined_score_1,combined_score_2,combined_score_3
5,24b87304a1eb4e95856da7f574a66fe8,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
16,677fbf669a0d496c843190c116b4f173,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333
34,b5e026a9cf014d71b110c58108a21752,0,1,0,0,0,0,0,0,1,1,0,0,12,1036,0.323529,0.404863,0.500000
36,be17b556785c428e8dabbef0c9d11f6f,1,0,0,0,1,0,0,0,0,1,1,1,135,2183,0.664706,0.853105,0.650000
41,d44d5fead16c47a986287ff2bc48d20f,0,1,0,0,0,0,1,0,1,1,1,1,47,1071,0.392157,0.418541,0.583333
43,d849ec19c3834c2b9c9ff2ab34d2f3e3,0,1,0,0,0,0,1,1,1,1,1,1,63,1087,0.423529,0.424794,0.600000
45,db873d7e50684080ba8f39aed56b288e,0,1,0,0,0,0,1,0,1,1,1,1,47,1071,0.392157,0.418541,0.583333
46,dd1000315e5b43ac89ae1d416400d9d6,0,1,0,0,0,0,1,1,1,1,1,1,63,1087,0.423529,0.424794,0.600000
49,e8a9dc7b6127482987d9b6dbca73ec5e,0,1,0,0,0,0,1,0,1,1,1,1,47,1071,0.392157,0.418541,0.583333
52,f7f404137f974160bcbfa423f5d35eda,0,1,0,0,0,0,0,0,1,0,0,0,8,1032,0.315686,0.403300,0.433333
